# Example 2: Fire weather in Portugal

This notebook shows how to reproduce `Ex2_fwixd_PRT_percentiles_for_GMT_levels.nc`

First steps:

1) Download the GMT trajectories of PROVIDE scenarios (https://zenodo.org/records/6833278)

2) Create fire weather emulations for GMT trajectories listed in `fair_mesmer_coupling.csv` using MESMER-X (https://zenodo.org/records/10300296)

In [1]:
import os,sys,glob,traceback, pickle
import pandas as pd
import xarray as xr
import numpy as np
import scipy
from scipy import stats
from scipy.special import erf
import matplotlib.pyplot as plt
from multiprocessing import Pool

In [ ]:
source = '/location/where/mesmer-x/emulations/are/stored/'
indicator = 'fwixd'
region = 'PRT'

scenario_dict = {
    'CurPol' : 'curpol-sap',
    'ModAct' : 'modact-sap',
    'Ren' : 'ren',
    'LD' : 'ld',
    'GS' : 'gs',
    'SP' : 'sp',
    'Neg' : 'neg-os-0',
    'ssp119' : 'ssp119-extended',
    'ssp534-over' : 'ssp534-over-extended',
    'Ref_1p5' : 'ref-1p5-extended',
}

# Table that shows which FaIR runs is used for which MESMER-X emulation
fair_mesmer_run = pd.read_csv('fair_mesmer_coupling.csv')

# see what ESMs are available
esms = np.unique([fl.split('_')[-6] for fl in\
     glob.glob(f'{source}/{indicator}/mesmerx_*_*_{indicator}_regional_averages_realisations.nc')])

In [ ]:
gmts = np.arange(0.0, 5., 0.1).round(1)
# match years to gmt bins
selected_years = {}
for scenario in scenarios:
    fair2mesmer = {k:v for k,v in zip(fair_mesmer_run[scenario].values, fair_mesmer_run[scenario].index)}
    mesmer2fair = {v:k for k,v in zip(fair_mesmer_run[scenario].values, fair_mesmer_run[scenario].index)}

    # identify runs that were used for mesmer emulations
    relevant_runs = sorted(fair2mesmer.keys())

    # load gmst trajectories from fair
    gmst = pd.read_csv(f'../fair_temperatures/scen_{scenario}.csv', index_col=0)
    gmst.columns = [int(c) for c in gmst.columns]

    # select gmst trajectories that correspond to the mesmer emulations
    gmst = gmst.loc[:,relevant_runs]
    gmst.columns = [fair2mesmer[c] for c in gmst.columns]

    # apply 21-year running mean
    gmst = gmst.rolling(window=21, center=True).mean()
    gmst = gmst.iloc[10:-10]
    gmst = gmst.melt(value_vars=gmst.columns, var_name='run', value_name='gmst', ignore_index=False)

    # construct dict of years that match gmst bin in given run
    selected_years[scenario] = {r:{g:list(gmst.loc[(gmst.run == r) & (np.abs(gmst.gmst - g) < 0.05)].index) \
                                    for g in gmts} for r in range(100)}


In [ ]:
def do_one_scenario(scenario):
    x = np.array([])
    # open dummy file and construct a gmt mask
    # the mask has the same dimensions as a mesmerX emulation file, but with the gmt in that year instead of the mesmerx value 
    fl = f'{source}/{indicator}/mesmerx_CanESM5_{scenario_dict[scenario]}_{indicator}.nc'
    y = xr.open_dataset(fl)[indicator].loc[:2100]

    gmt_mask = y.copy() * np.nan
    for run in range(100):
        years = np.array(selected_years[scenario][run][gmt])
        years = years[years <= 2100]
        gmt_mask.loc[years,run,:] = gmt

    # go through esms, open file and store all values corresponding to the gmt (global vriable) in x
    for esm in esms:
        fl = f'{source}/{indicator}/mesmerx_{esm}_{scenario_name_mesmerX}_{indicator}.nc'
        y = xr.open_dataset(fl)[indicator].loc[:2100,'PRT'].values

        # calculate are average over Portugal
        reg_mask = regionmask.defined_regions.natural_earth_v5_0_0.countries_110.mask_3D_frac_approx(y.lon, y.lat)
        reg_mask = reg_mask[reg_mask.names == 'Russia'].squeeze()
        reg_mask *= np.repeat(np.cos(np.radians(reg_mask.lat.values)), reg_mask.shape[1]).reshape(reg_mask.shape)
        reg_mask /= reg_mask.sum()

        x = np.append(x, y[gmt_mask == gmt].flatten())
    # save the values
    np.savetxt(f"Ex2_tmp/{gmt}_{scenario}.csv", x, delimiter=",")  

In [ ]:
for gmt in gmts:
    with Pool(10) as p:
        pool_outputs = list(p.map(do_one_scenario, scenario_dict.keys()))
    
# construct an array with 100 percentiles for each gmt level
percentiles = xr.DataArray(dims=['gmt','p'], coords=dict(gmt=gmts, p=np.arange(0,101,1,'int')))
for gmt in gmts:
    x = np.array([])
    for scenario in scenario_dict.keys():
        x = np.append(x, np.loadtxt(f"Ex2_tmp/{gmt}_{scenario}.csv", delimiter=","))
    percentiles.loc[gmt][:] = np.percentile(x, range(101))

xr.Dataset({indicator:percentiles}).to_netcdf(f'Ex2_fwixd_PRT_percentiles_for_GMT_levels.nc')